## Import Libraries

In [109]:
# Data handling
import pandas as pd
import numpy as np
import yfinance as yf
from datetime import datetime, timedelta
import requests
from tqdm import tqdm
import os
import time
import json
from bs4 import BeautifulSoup
import requests
# PyTorch ecosystem
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, TensorDataset
# Preprocessing & metrics
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.model_selection import train_test_split, TimeSeriesSplit
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
# Suppress warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings('ignore')


## Process Data

#### Collect tickers (archaic)

In [110]:
## Import a list of tickers to include in the dataset
# # Wikipedia URL for S&P 500 Tickers
url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'}
response = requests.get(url, headers=headers)
# Pull the table from Wikipedia
sp500_tables = pd.read_html(response.text)
# Figure out a way to only keep the "Symbols" column
sp500_df = sp500_tables[1]
tickers = sp500_df['Symbol'].str.replace('.', '-').to_list()

#### Access SimFin API

In [111]:
import simfin as sf
from simfin.names import *

# FinancialModelingPrep API Key
sf.set_api_key('3710f0ef-cd08-4273-ad23-6caf78ea7396')

# Set local data directory
DATA_DIR = '/home/lukeditzler/projects/pytorch/examples/Financial_Markets_Model/data/simfin_data/'
os.makedirs(DATA_DIR, exist_ok=True)

# Set local data directory (where bulk downloads will be stored)
sf.set_data_dir('simfin_data')


In [112]:
# --- 1. Load quarterly statements ---
df_balance_q = sf.load_balance(variant='quarterly', market='us')
df_income_q = sf.load_income(variant='quarterly', market='us')
df_cash_q = sf.load_cashflow(variant='quarterly', market='us')

# Bring index into columns
df_balance_q = df_balance_q.reset_index()
df_income_q = df_income_q.reset_index()
df_cash_q = df_cash_q.reset_index()

# --- 2. Load daily share prices ---
df_prices = sf.load_shareprices(variant='daily', market='us')
df_prices = df_prices.reset_index()


Dataset "us-balance-quarterly" on disk (0 days old).
- Loading from disk ... Done!
Dataset "us-income-quarterly" on disk (0 days old).
- Loading from disk ... Done!
Dataset "us-cashflow-quarterly" on disk (0 days old).
- Loading from disk ... Done!
Dataset "us-shareprices-daily" on disk (0 days old).
- Loading from disk ... Done!


#### Define Functions

In [116]:
# ----------------------------------------------------------
# 1. Merge quarterly fundamentals
# ----------------------------------------------------------
def merge_quarterly_fundamentals(df_income_q, df_balance_q, df_cash_q):
    start = time.perf_counter()
    print("\n--- MERGING QUARTERLY FUNDAMENTALS ---")

    key_cols = ["Ticker", "Report Date"]

    # Ensure datetime for merge consistency
    for df in [df_income_q, df_balance_q, df_cash_q]:
        df["Report Date"] = pd.to_datetime(df["Report Date"])

    # Merge
    fundamentals = (
        df_income_q.merge(df_balance_q, on=key_cols, how="left")
                   .merge(df_cash_q, on=key_cols, how="left")
    )

    print(f"Before cleaning shape: {fundamentals.shape}")

    # --- CLEAN DUPLICATE SUFFIX COLS HERE ---
    fundamentals = drop_suffix_duplicate_columns(fundamentals)

    elapsed = time.perf_counter() - start
    print(f"Fundamentals merged in {elapsed:.2f} seconds")
    print(f"After cleaning shape: {fundamentals.shape}")

    return fundamentals

def drop_suffix_duplicate_columns(df):
    """
    Keeps clean column names (without suffixes) and removes all suffixed versions
    like `_x`, `_y`, `_BAL`, `_CASH`, etc.
    
    If no clean version exists (rare), one suffixed version is kept.
    """
    cols_to_drop = []
    keep_cols = set()

    # Identify base names and available clean columns
    base_to_clean = {}
    base_to_suffixes = {}

    for col in df.columns:
        parts = col.split('_')
        
        # Heuristic: suffixes are short (x,y,BAL,CASH)
        is_suffix = len(parts) > 1 and parts[-1].upper() in {"X", "Y", "BAL", "CASH"}

        base = parts[0]

        if not is_suffix:
            base_to_clean[base] = col
        else:
            base_to_suffixes.setdefault(base, []).append(col)

    # Decide what to drop
    for base, suffixed_cols in base_to_suffixes.items():
        if base in base_to_clean:
            # Clean column exists → drop *all* suffixed versions
            cols_to_drop.extend(suffixed_cols)
        else:
            # No clean version → keep ONE suffixed version, drop the rest
            keep_cols.add(suffixed_cols[0])
            cols_to_drop.extend(suffixed_cols[1:])

    df = df.drop(columns=cols_to_drop, errors='ignore')

    print("\nDropped duplicate-suffix columns:")
    for c in cols_to_drop:
        print("  -", c)

    return df

# ----------------------------------------------------------
# 2. Merge fundamentals with daily prices (corrected)
# ----------------------------------------------------------
def merge_fundamentals_with_prices(fundamentals, df_prices):
    start = time.perf_counter()
    print("\n--- MERGING FUNDAMENTALS + PRICES ---")

    # Convert dates
    fundamentals["Report Date"] = pd.to_datetime(fundamentals["Report Date"])
    df_prices["Date"] = pd.to_datetime(df_prices["Date"])

    # Sort both DataFrames
    fundamentals = fundamentals.sort_values(['Ticker', 'Report Date']).reset_index(drop=True)
    df_prices = df_prices.sort_values(['Ticker', 'Date']).reset_index(drop=True)

    # Merge per ticker using merge_asof
    merged_list = []
    for ticker in fundamentals['Ticker'].unique():
        fund_ticker = fundamentals[fundamentals['Ticker'] == ticker].copy()
        price_ticker = df_prices[df_prices['Ticker'] == ticker].copy()

        merged = pd.merge_asof(
            fund_ticker,
            price_ticker,
            left_on='Report Date',
            right_on='Date',
            direction='backward'
        )
        merged_list.append(merged)

    master = pd.concat(merged_list, ignore_index=True)

    # Optional: rename price date for clarity
    master = master.rename(columns={'Date':'Price Date'})

    elapsed = time.perf_counter() - start
    print(f"Master DF merged in {elapsed:.2f} seconds")
    print(f"Master DF shape: {master.shape}")

    return master


# ----------------------------------------------------------
# 3. Drop sparse columns
# ----------------------------------------------------------
def drop_sparse_columns(df, min_non_null_pct=0.50): # default 50% data
    start = time.perf_counter()
    print("\n--- DROPPING SPARSE COLUMNS ---")

    threshold = len(df) * min_non_null_pct

    # Columns above threshold → keep
    good_cols = [c for c in df.columns if df[c].count() >= threshold]

    # Columns below threshold → drop
    dropped_cols = [c for c in df.columns if c not in good_cols]

    df_clean = df[good_cols]

    elapsed = time.perf_counter() - start
    print(f"Sparse-column cleanup completed in {elapsed:.2f} seconds")
    print(f"Before: {df.shape} → After: {df_clean.shape}")

    if dropped_cols:
        print("\nDropped sparse columns:")
        for col in dropped_cols:
            print(f"  - {col}")
    else:
        print("No columns dropped.")


        df_balance_q,
    df_cash_q

    # Remove columns that are mostly descriptive/metadata
    df_clean = df_clean.rename(columns={'Ticker_x': 'Ticker'})
    meta_cols = ['Ticker_y','SimFinId_x','SimFinId_y', "Currency","Fiscal Year","Fiscal Period",
                "Publish Date","Restated Date"]
    df_clean = df_clean.drop(columns=[c for c in meta_cols if c in df_clean.columns])

    # Remove rows that didn't connect to price data (~4%)
    price_cols = ['Open','High','Low','Close','Adj. Close','Volume','Shares Outstanding']
    df_clean = df_clean.dropna(subset=price_cols)

    return df_clean


#### Run Script

In [ ]:
# Combine Income Statement, Balance Sheet, and Cash Flow DataFrames
fundamentals = merge_quarterly_fundamentals(
    df_income_q,
    df_balance_q,
    df_cash_q
)

# Align each quarterly report with the last available stock price
master_df = merge_fundamentals_with_prices(
    fundamentals,
    df_prices
)

# Remove any columns where less than 75% of the rows have data
master_df = drop_sparse_columns(master_df, min_non_null_pct=0.75)

# Save to CSV
master_df.to_csv("historical_finance_data.csv", index=False)


--- MERGING QUARTERLY FUNDAMENTALS ---
Before cleaning shape: (53186, 82)

Dropped duplicate-suffix columns:
  - SimFinId_x
  - SimFinId_y
  - Currency_x
  - Currency_y
  - Fiscal Year_x
  - Fiscal Year_y
  - Fiscal Period_x
  - Fiscal Period_y
  - Publish Date_x
  - Publish Date_y
  - Restated Date_x
  - Restated Date_y
  - Shares (Basic)_x
  - Shares (Basic)_y
  - Shares (Diluted)_x
  - Shares (Diluted)_y
  - Depreciation & Amortization_y
Fundamentals merged in 0.05 seconds
After cleaning shape: (53186, 65)

--- MERGING FUNDAMENTALS + PRICES ---
Master DF merged in 489.27 seconds
Master DF shape: (53186, 76)
